In [0]:
import keras
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
from itertools import count
from sklearn.metrics import accuracy_score
from keras.datasets import mnist
from keras.applications.vgg16 import VGG16
from keras.layers import Dense, Dropout, Flatten, Activation, Input, Conv2D, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D
from sklearn.model_selection import train_test_split
from keras.models import Model
from keras.callbacks import EarlyStopping
from scipy.stats import pearsonr
from tqdm import tqdm

In [0]:
BATCH_SIZE = 128
EPOCHS = 9999
IMAGE_SIZE = 28
NUM_CLASSES = 10
MODEL_NAME = "MNIST_baseline"
PATH = ""

# Set seeds

In [0]:
np.random.seed(1)
tf.random.set_seed(1)

# Preprocess

In [0]:
def preprocess(imgs):
    
    return imgs.reshape(imgs.shape[0], IMAGE_SIZE, IMAGE_SIZE, 1)

In [16]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = preprocess(x_train)
x_test = preprocess(x_test)

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [0]:
# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, NUM_CLASSES)
y_testc = keras.utils.to_categorical(y_test, NUM_CLASSES)

In [0]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [0]:
# Split the data
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.20, shuffle= True)

# Model

In [0]:
def MNISTmodel(imsize, num_classes, num_channels):
    inputs = Input((imsize,imsize,num_channels))
    x = Conv2D(filters=16, kernel_size=(3,3), activation='relu',strides = 2)(inputs)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same')(x)
    x = Conv2D(filters=16, kernel_size=(1,1), activation='relu', padding='valid')(x)
    x = Conv2D(filters=10, kernel_size=(1,1),strides=(1,1), padding='valid')(x)
    x = GlobalAveragePooling2D()(x)
    outputs = Activation('softmax')(x)
    
    model = Model(inputs=inputs, outputs=outputs)

    optimizer = keras.optimizers.Adam(learning_rate = 1e-04)

    model.compile(loss='categorical_crossentropy',
                      optimizer=optimizer,
                      metrics=['accuracy'])
    return model

# Predict

In [0]:
def predict(model, X, Y):
    return accuracy_score(np.argmax(model.predict(X), axis=1), np.argmax(Y, axis=1))

# Train

In [23]:
model = MNISTmodel(IMAGE_SIZE, NUM_CLASSES, 1)

es = EarlyStopping(min_delta=0.01, patience=3)
model.fit(x_train,y_train,
          batch_size = BATCH_SIZE,
          epochs = EPOCHS,
          validation_data = (x_test,y_testc),
          shuffle = True,
          callbacks=[es])

model.save_weights(PATH + MODEL_NAME + f"_weights.h5" )

Train on 48000 samples, validate on 10000 samples
Epoch 1/5
48000/48000 [==============================] - 2s 48us/step - loss: 2.3108 - accuracy: 0.1360 - val_loss: 2.2860 - val_accuracy: 0.2087
Epoch 2/5
48000/48000 [==============================] - 2s 43us/step - loss: 2.2411 - accuracy: 0.2254 - val_loss: 2.2188 - val_accuracy: 0.2525
Epoch 3/5
48000/48000 [==============================] - 2s 42us/step - loss: 2.1866 - accuracy: 0.2810 - val_loss: 2.1500 - val_accuracy: 0.2878
Epoch 4/5
48000/48000 [==============================] - 2s 42us/step - loss: 2.1209 - accuracy: 0.3129 - val_loss: 2.0764 - val_accuracy: 0.3185
Epoch 5/5
48000/48000 [==============================] - 2s 42us/step - loss: 2.0441 - accuracy: 0.3244 - val_loss: 1.9953 - val_accuracy: 0.3272


# Results

## Accuracy
The final accuracy of the ensamble on the test set

In [29]:
print("Accuracy: " + str(predict(model, x_test, y_testc)))

Accuracy: 0.3272
